# Investigation of Similarity measures
### Import precomp libs


In [1]:
#import the spacy english model
# "python -m spacy download en"   to download english models
import spacy
nlp = spacy.load('en') # this should take some time like 10s to load
import numpy as np
import scipy as sp
import pandas as pd
import time
from datetime import datetime

SKIP to the block of reading in SpaCy objects

In [2]:
# PROJECT structure: data directory with the two csvs of train,test and the ipython notbook in its own directory
# loading the data
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')
print train.shape, test.shape

(404290, 6) (2345796, 3)


In [7]:
train.head(10)

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1
6,6,13,14,Should I buy tiago?,What keeps childern active and far from phone ...,0
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1
8,8,17,18,When do you use シ instead of し?,"When do you use ""&"" instead of ""and""?",0
9,9,19,20,Motorola (company): Can I hack my Charter Moto...,How do I hack Motorola DCX3400 for free internet?,0


In [8]:
#this procedure will be mapped to each row
def eval_sysim(row):
    q1_txt = nlp(unicode(row['question1']))
    q2_txt = nlp(unicode(row['question2']))
    # feature: similarity measure from built-in spacy
    #word net similarity? or hamming distance of the strings?
    sy_sim = q1_txt.similarity(q2_txt)

    #feature: 
    return sy_sim

def u_eval_test(fun):
    #unit test(s)
    utest = train.loc[1]
    print fun(utest)
    utest = train.loc[3]
    print fun(utest)
    utest = train.loc[9]
    print fun(utest)
    


def eval_sysim1(row):
    q1_txt = nlp(unicode(row['question1']))
    q2_txt = nlp(unicode(row['question2']))
    q1 = []
    q2 = []
    for w1 in q1_txt:
        if (w1.is_stop == False):
            q1.append(w1)
    for w2 in q2_txt:
        if (w2.is_stop == False):
            q2.append(w1)
    # feature: similarity measure from built-in spacy
    #word net similarity? or hamming distance of the strings?
    q1 = nlp(q1)
    q2 = nlp(q2)
    sim = 0.0
    
    reduce(lambda x,y: x+y, )
    sy_sim = q1.similarity(q2)

    #feature: 
    return sy_sim

u_eval_test(eval_sysim)
#u_eval_test(eval_sysim1)
    

0.934445645664
0.740894391357
0.819441244603


In [9]:
def jacc_sim(row):
    q1_txt = nlp(unicode(row['question1']))
    q2_txt = nlp(unicode(row['question2']))
    #test jacc sim
    q1 = set()
    q2 = set()
    for w in q1_txt:
        if w.pos_ in ['NOUN','PROPN','VERB']:
            q1.add((w.text, w.pos_))
    for w in q2_txt:
        if w.pos_ in ['NOUN','PROPN','VERB']:
            q2.add((w.text, w.pos_))
    return len(q1.intersection(q2)) / (1.0*len(q1.union(q2)))   

print jacc_sim(train.loc[7])

0.333333333333


In [10]:
from nltk.corpus import wordnet as wn
#IMPORTANCE OF ENTITIES NOT IN BOTH
diff_ents = []
def diff_ents(row):
    q1_txt = nlp(unicode(row['question1']))
    q2_txt = nlp(unicode(row['question2']))
    #test jacc sim
    q1 = set()
    q2 = set()
    q1_n = []
    q2_n = []
    for w in q1_txt.ents:
        q1.add(w)
    for w in q2_txt.ents:
        q2.add(w)
    for n in q1_txt.noun_chunks:
        q1_n.append(n)
    for n in q2_txt.noun_chunks:
        q2_n.append(n)    
    return (q1,q2,q1_n,q2_n) 

a,b,c,d = diff_ents(train.loc[5])
c = map(lambda x: x.text.split(), c)
c = [item for sublist in c for item in sublist]
d = map(lambda x: x.text.split(), d)
d = [item for sublist in d for item in sublist]
rez = []
for word1 in c:
    for word2 in d:
        wordFromList1 = wn.synsets(word1)
        wordFromList2 = wn.synsets(word2)
        if wordFromList1 and wordFromList2: #Thanks to @alexis' note
            s = wordFromList1[0].wup_similarity(wordFromList2[0])
            rez.append(s)
reduce(lambda x,y: x+y, rez)/len(rez)

0.3205414059688226

here we see the pitfall in relying on the similarity measure of the two doccuments as their scope differs(invest in shares vs invest in shares of **INDIA**). To extend this baseline model we will look at parts of speech and entity recognition to help parse scope.

We can however use the similarity as the first filter to remove completely unrelated questions


In [3]:
from nltk.corpus import wordnet as wn

def wn_sim(q1doc,q2doc):
    for word1 in q1doc:
        max_w1 = 0.0
        for word2 in q2doc:
            wordFromList1 = wn.synsets(word1.text)
            wordFromList2 = wn.synsets(word2.text)
            if wordFromList1 and wordFromList2:
                s = wordFromList1[0].wup_similarity(wordFromList2[0])
                if (s > max_w1):
                    max_w1 = s
            rez.append(max_w1)
    return reduce(lambda x,y: x+y, rez)/len(rez)

In [12]:
from nltk.corpus import stopwords
objts = []

def parse_clean(row):
    q1_txt = [word for word in row['question1'].split() if word not in stopwords.words('english')]
    q2_txt = [word for word in row['question2'].split() if word not in stopwords.words('english')]
    return (q1_txt,q2_txt)

from nltk.tokenize import word_tokenize
def tokenize(sent):
    tokens = word_tokenize(sent)
    return tokens

def parse_clean(row):
    q1_txt = [word for word in row['question1'].split() if word not in stopwords.words('english')]
    q2_txt = [word for word in row['question2'].split() if word not in stopwords.words('english')]
    return (q1_txt,q2_txt)

import nltk
def pos_tag(tokens):
    return nltk.pos_tag(tokens)


from nltk.stem.porter import PorterStemmer
def stem(tokens):
    pstem = PorterStemmer()
    return map(lambda x: pstem.stem(x),tokens)


from nltk.stem.wordnet import WordNetLemmatizer
def word_stem(tokens):
    rez = []
    for tok in tokens:
        tokdk = nlp(unicode(tok))
        pos = ""
        for x in tokdk:
            pos = x.pos_
        pstem = WordNetLemmatizer()
        print pos
        rez.append(pstem.lemmatize(tok))
    return rez 



I will investigate regression on the POS tagging and have one feature be lemmatized verb similarity and entity similarites with penalty for entities not encluded

## in a different file "train_wrod2vec.py"

In [4]:
from gensim.models import word2vec

model = word2vec.Word2Vec.load("saved_vec_mod.kf")

def vec_diff(doc1,doc2):
    a = doc1.vector
    b = doc2.vector
    return a - b
    
def wmdist(sent1,sent2):
    """This requires that the parameters @sent1 @sent2 are lists of strings ideally
    lemmatized and cleaned sentences from above"""
    return model.wmdistance(sent1, sent2)
  

In [5]:
vec_diff(nlp(unicode(train.loc[2]['question1'])),nlp(unicode(train.loc[2]['question2'])))

array([  1.84825569e-01,  -1.45079494e-02,  -1.18102714e-01,
         2.24120393e-02,   4.02033962e-02,   7.42071047e-02,
        -5.29460944e-02,  -8.10047314e-02,   4.59638089e-02,
         8.38956833e-02,  -9.13993120e-02,   1.98517311e-02,
         1.57576144e-01,  -3.26649062e-02,   6.07710928e-02,
         2.86095738e-02,  -3.31799015e-02,   1.80741787e-01,
        -9.44152623e-02,   4.09685075e-03,  -1.10343508e-02,
         6.92504421e-02,  -3.26047540e-02,   1.95322931e-02,
         2.66033038e-03,  -7.41237476e-02,  -3.12730931e-02,
        -1.13286287e-01,  -3.75003293e-02,   8.37820172e-02,
         6.05418310e-02,   9.34930742e-02,   4.99187037e-02,
         4.36130837e-02,   1.34942234e-02,  -2.58105472e-02,
         5.46048619e-02,  -1.78474814e-01,  -1.36214584e-01,
        -8.10364038e-02,  -9.23259743e-03,   1.03099123e-02,
        -7.78148472e-02,  -1.07806772e-02,   6.82395548e-02,
        -4.94703203e-02,   1.57432109e-02,   4.07735258e-02,
        -3.39489877e-02,

In [6]:
wmdist(pipe(nlp(unicode(train.loc[2]['question1']))), pipe(nlp(unicode(train.loc[2]['question2']))))

NameError: name 'pipe' is not defined

In [27]:
len(train_docs)

NameError: name 'train_docs' is not defined

In [7]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

def pipe(question):
    """ NEEDS A SPACY DOC OF THE QUESTION"""
    b = [x for x in question if not x.is_punct]
    a = [x.lemma_ if not x.lemma_ == '-PRON-' else x.text for x in b ]
    return [x for x in a if x not in stop]

pipe(nlp(unicode(train.loc[2]['question1'])))

[u'I', u'increase', u'speed', u'internet', u'connection', u'use', u'vpn']

In [8]:
def clean(doc):
    b = [x for x in doc if not x.is_punct]
    a = [x.lemma_ if not x.lemma_ == '-PRON-' else x.text for x in b ]
    return nlp(u''.join([x + ' ' for x in a if x not in stop]))
clean(nlp(unicode(train.loc[2]['question1'])))

I increase speed internet connection use vpn 

READ THE DATA and then train model!

In [9]:
test_loc = 'test_dump.bin'
train_loc = 'train_dump.bin'


from spacy.tokens.doc import Doc
test_docs = []
train_docs = []
i = 0
#print datetime.now().strftime('%Y-%m-%d %H:%M:%S')
with open(train_loc, 'rb') as file_:
    for byte_string in Doc.read_bytes(file_):
        if i%100000 == 0: print i, datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        train_docs.append(clean(Doc(nlp.vocab).from_bytes(byte_string)))
        i += 1

0 2017-05-14 15:27:25
100000 2017-05-14 15:28:15
200000 2017-05-14 15:28:59
300000 2017-05-14 15:29:42
400000 2017-05-14 15:30:26
500000 2017-05-14 15:31:08
600000 2017-05-14 15:31:51
700000 2017-05-14 15:32:32
800000 2017-05-14 15:33:15


In [11]:
d = []
ct = 0
for q1,q2 in [(train_docs[2*i],train_docs[2*i+1]) for i in range(len(train_docs)/2)]:
    d.append({
              'sysim': q1.similarity(q2), # spacy sim
#              'netsim':  wn_sim(q1c,q2c), #float sim by wordnet
              'wmd': wmdist([x.text for x in q1], [x.text for x in q2]), # returns float dist
              'vectdiff': vec_diff(q1,q2) # 300 dim vect
    })
    if ct % 10000 == 0: print ct, datetime.now().strftime('%H:%M:%S')
    ct = ct+1    
features = pd.DataFrame(d)
features.to_csv('features.csv')

0 15:34:00
10000 15:34:10
20000 15:34:21
30000 15:34:32
40000 15:34:42
50000 15:34:54
60000 15:35:05
70000 15:35:15
80000 15:35:26
90000 15:35:37
100000 15:35:47
110000 15:35:58
120000 15:36:08
130000 15:36:19
140000 15:36:30
150000 15:36:43
160000 15:36:54
170000 15:37:05
180000 15:37:15
190000 15:37:26
200000 15:37:37
210000 15:37:47
220000 15:37:58
230000 15:38:08
240000 15:38:19
250000 15:38:30
260000 15:38:40
270000 15:38:54
280000 15:39:04
290000 15:39:16
300000 15:39:26
310000 15:39:37
320000 15:39:48
330000 15:39:59
340000 15:40:10
350000 15:40:20
360000 15:40:31
370000 15:40:42
380000 15:40:53
390000 15:41:05
400000 15:41:16


Now that we have all the SpaCy docs loaded in, we will generate features and run a Regression on them.
The methods we will use are:
- logistic regression
- least squares
- svm
- SGD